In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
from scipy import stats
import re
from scipy.stats.stats import pearsonr
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import copy
from collections import Counter
import seaborn as sns
import os
from scipy.stats import binom_test
from scipy.stats import mannwhitneyu as mwu

#Try to figure out what the characteristics are of promoters that have matching directionality!
#But first, we need to filter the peaks!
#Lets just filter so that they have to have matching direction in replicates and humreffed/chpreffed if they are to be called
#as significant
def match_dir(a, b, cutoff_sig = 0.5):
    if abs(a) > cutoff_sig or abs(b) > cutoff_sig:
        if a >= 0 and b >= 0 or a <= 0 and b <= 0:
            return True
        else:
            return False
    return True

def bias_check(a, b):
    if abs(a-b) > 1:
        return True
    else:
        return False
    
def map_bias_check(a, b):
    if abs(np.log2(a/b)) > 2:
        return False
    else:
        return True
    
def mean_rat(file, v):
    if "SKM" not in file and "HP" not in file:
        v["Mean Ratio"] = np.log2(((v[4] + v[6])/(v[5] + v[7]) + (v[9] + v[11])/(v[10] + v[12]))/2)
    else:
        v["Mean Ratio"] = np.log2(((v[4]/v[5]) + (v[7]/v[8]))/2)
    return v

#Identify cell type specific peaks
def unique_ct(ct, l, name):
    if ct in name:
        for i in l:
            if i in name:
                return False
        return True
    else:
        return False
def ubiq(l, name, cut):
    c = 0
    for i in l:
        if i in name:
            c += 1
    if c >= cut:
        return True
    else:
        return False
    
#Read in the files, can adjust conditions in first if statement to determine which are most important.
def readin(cell, reffed, folder):
    files = os.listdir(folder)
    cols = ["Peak"]
    d = {}
    for f in files:
        if cell in f and reffed in f and ".txt" in f:
            cols.append(f)
            v = pd.read_csv(folder+ "/" + f, sep = "\t", header = None)
            for index, row in v.iterrows():
                if "eak" in row[0]:
                    if row[0] in d.keys():
                        d[row[0]].append(row[1])
                    else:
                        d[row[0]] = [row[1]]
    to_df = []
    for key in d.keys():
        to_df.append([key] + d[key])
    df = pd.DataFrame(to_df)
    df.columns = cols
    return df

#vv = pd.read_csv("MN" + "_ATAC_Filtered_SNP_Init.txt", sep = '\t')
def addgene(v, t):
    o = []
    for index, row in v.iterrows():
        l = row["Peak Chpreffed"]
        try:
            if t == "CM": 
                if "hancer" in l:
                    gene = l.split("_")[7]
                    o.append(list(row) + [gene])
                else:
                    gene = l.split("_")[7]
                    o.append(list(row) + [gene])
            else:
                if "hancer" in l:
                    gene = l.split("_")[6]
                    o.append(list(row) + [gene])
                else:
                    gene = l.split("_")[6]
                    o.append(list(row) + [gene])
        except:
            pass
    df = pd.DataFrame(o)
    df.columns = list(v.columns) + ["gene"]
    return df
#vvv = addgene(vv, "MN")

def txt_to_bed(txt):
    v = pd.read_csv(txt, sep = "\t")
    out = []
    for index, row in v.iterrows():
        g = row["Genomic location"]
        l = [g.split(":")[0], g.split(":")[1].split("-")[0], g.split(":")[1].split("-")[1]] + list(row)
        out.append(l)
    df = pd.DataFrame(out)
    df.to_csv(txt.replace(".txt", ".bed"), sep = "\t", header = False, index = False)

In [12]:
#Amalgamate the counts for uploading to GSE
df = 0
ind = 1
x = 'ATAC7_CM_Hyb1_total'
d = {"Chimp":"chimp", "Human":"human", "BothAlleles":"total"}
for file in os.listdir('All_Peaks'):
    if "humreffed_all_peaks.txt" in file:
        v = pd.read_csv("All_Peaks/" + file, sep = '\t', header = None).set_index(0)
        spec = d[file.split("_")[3]]
        start_string = "_".join(file.split("_")[0:3])
        v.columns = [start_string + "_" + spec]
        #print(v.columns)
        if ind:
            df = v
            ind = 0
        else:
            df = df.join(v)
#df.to_csv("ATAC_Counts_Table_Human_Aligned.txt", sep = "\t")

In [11]:
#Processing the striatal organoid data for plotting
v = pd.read_csv("GSE132403_Data_03_all_peak_counts_updated.tsv", sep = "\t").set_index("Peak").T
out = []
def cpm(x):
    s = sum(list(x))
    x = [i*1000000/s for i in list(x)]
    return x

for index, row in v.iterrows():
    out.append(cpm(row))
v_cpm = pd.DataFrame(out).T

v_cpm.index = v.T.index
v_cpm.columns = v.T.columns

v_cpmf = pd.DataFrame(v_cpm.loc["peak_12384"])
out = []
for index, row in v_cpmf.iterrows():
    out.append([row["peak_12384"], index.split("_")[1], "D" + index.split("_")[2], index.split("_")[4], index.split("_")[3]])
df = pd.DataFrame(out)
df.columns = ["CPM", "Cell Type", "Day", "Organoid", "Line"]

df.to_csv("GAD1_HAR_Containing_Peak_ATAC_CPM.txt", sep = "\t")

In [ ]:
#Function to perform peak filtering and compute binomil p-values when we have replicates
#Ignore the "interpeak" parts as these were unused
fix_all_peaks = {"peak__peakMN86472387":"peak_MN_64043", "peak_HP_MN_PP_SKM4.96014":"peak_HP_MN_PP_SKM_78155", "peak_HP_MN_PP_SKM4.944030000000001":"peak_HP_MN_PP_SKM_77849", "peak_HP_MN_PP_SKM4.83314":"peak_HP_MN_PP_SKM_268883", "peak_HP_MN_PP_SKM4.73749":"peak_HP_MN_PP_SKM_179978"}
fix_all_peaks_down = {"peak_HP_MN_PP_SKM25.1425":"peak_HP_MN_PP_35800", "peak_HP_MN_PP_SKM36.5991":"peak_HP_MN_PP_SKM_68285"}

def filter_rep(ct="CM", fold="All_Peaks", mapping={"S1":"ATAC7", "S2":"ATAC8"}, suffix = "_all_peaks.txt"):
    df_hum = readin(ct, "umreffed", fold)
    df_chp = readin(ct, "hpreffed", fold)
    if fold == "All_Peaks" or fold == "Down":
        indices = []
        for index, row in df_hum.iterrows():
            if "promoter" in row["Peak"]:
                ad = row["Peak"].split("promoter")[0][:-1]
            else:
                ad = row["Peak"].split("enhancer")[0][:-1]
            indices.append(ad)
        df_hum.index = indices

        indices = []
        for index, row in df_chp.iterrows():
            if "promoter" in row["Peak"]:
                ad = row["Peak"].split("promoter")[0][:-1]
            else:
                ad = row["Peak"].split("enhancer")[0][:-1]
            indices.append(ad)
        df_chp.index = indices

        c = list(df_hum.columns)
        c[0] = "Peak Humreffed"
        df_hum.columns = c
        c = list(df_chp.columns)
        c[0] = "Peak Chpreffed"
        df_chp.columns = c
        df = df_hum.join(df_chp)
    else:
        df_hum  = df_hum.set_index("Peak")
        df_chp = df_chp.set_index("Peak")
        df = df_hum.join(df_chp)
        df["Peak"] = df.index
    df[mapping["S1"] + "_Human_Humr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix])
    df[mapping["S1"] + "_Chimp_Humr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix])
    df[mapping["S2"] + "_Human_Humr_CPM"] = 1000000*(df[mapping["S2"] + "_" + ct + "_Hyb2_Human_humreffed" + suffix]+1)/np.sum(df[mapping["S2"] + "_" + ct + "_Hyb2_Human_humreffed" + suffix])
    df[mapping["S2"] + "_Chimp_Humr_CPM"] = 1000000*(df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_humreffed" + suffix]+1)/np.sum(df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_humreffed" + suffix])
    df[mapping["S1"] + "_Human_Chpr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix])
    df[mapping["S1"] + "_Chimp_Chpr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix])
    df[mapping["S2"] + "_Human_Chpr_CPM"] = 1000000*(df[mapping["S2"] + "_" + ct + "_Hyb2_Human_chpreffed" + suffix]+1)/np.sum(df[mapping["S2"] + "_" + ct + "_Hyb2_Human_chpreffed" + suffix])
    df[mapping["S2"] + "_Chimp_Chpr_CPM"] = 1000000*(df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_chpreffed" + suffix]+1)/np.sum(df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_chpreffed" + suffix])

    df["Total_Counts_Humr"] = df[mapping["S1"] + "_Human_Humr_CPM"] + df[mapping["S1"] + "_Chimp_Humr_CPM"] + df[mapping["S2"] + "_Human_Humr_CPM"] + df[mapping["S2"] + "_Chimp_Humr_CPM"]
    df["Total_Counts_Chpr"] = df[mapping["S1"] + "_Human_Chpr_CPM"] + df[mapping["S1"] + "_Chimp_Chpr_CPM"] + df[mapping["S2"] + "_Human_Chpr_CPM"] + df[mapping["S2"] + "_Chimp_Chpr_CPM"]
    df[mapping["S1"] + "_fc_Humr"] = df[mapping["S1"] + "_Human_Humr_CPM"]/df[mapping["S1"] + "_Chimp_Humr_CPM"]
    df[mapping["S1"] + "_fc_Chpr"] = df[mapping["S1"] + "_Human_Chpr_CPM"]/df[mapping["S1"] + "_Chimp_Chpr_CPM"]
    df[mapping["S2"] + "_fc_Humr"] = df[mapping["S2"] + "_Human_Humr_CPM"]/df[mapping["S2"] + "_Chimp_Humr_CPM"]
    df[mapping["S2"] + "_fc_Chpr"] = df[mapping["S2"] + "_Human_Chpr_CPM"]/df[mapping["S2"] + "_Chimp_Chpr_CPM"]
    if ct == "PP":
        df["l2fc_Humr"] = np.log2(df[mapping["S1"] + "_fc_Humr"])
        df["l2fc_Chpr"] = np.log2(df[mapping["S1"] + "_fc_Chpr"])
    else:
        df["l2fc_Humr"] = np.log2((df[mapping["S1"] + "_fc_Humr"] + df[mapping["S2"] + "_fc_Humr"])/2)
        df["l2fc_Chpr"] = np.log2((df[mapping["S1"] + "_fc_Chpr"] + df[mapping["S2"] + "_fc_Chpr"])/2)
    df["mean_l2fc"] = (df["l2fc_Humr"] + df["l2fc_Chpr"])/2

    df["mean_counts_chimp_humr"] = (df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix] + df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_humreffed" + suffix])/2
    df["mean_counts_human_humr"] = (df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix] + df[mapping["S2"] + "_" + ct + "_Hyb2_Human_humreffed" + suffix])/2
    df["mean_counts_chimp_chpr"] = (df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix] + df[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_chpreffed" + suffix])/2
    df["mean_counts_human_chpr"] = (df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix] + df[mapping["S2"] + "_" + ct + "_Hyb2_Human_chpreffed" + suffix])/2

    out = []
    for index, row in df.iterrows():
        if (row["mean_counts_human_humr"] > 25 and row["mean_counts_human_chpr"] > 25) or (row["mean_counts_chimp_humr"] > 25 and row["mean_counts_chimp_chpr"] > 25):
            if not bias_check(np.log2(row[mapping["S1"] + "_fc_Humr"]), np.log2(row[mapping["S1"] + "_fc_Chpr"])) and not bias_check(np.log2(row[mapping["S2"] + "_fc_Humr"]), np.log2(row[mapping["S2"] + "_fc_Chpr"])):
                if match_dir(np.log2(row[mapping["S1"] + "_fc_Humr"]), np.log2(row[mapping["S2"] + "_fc_Humr"])) and match_dir(np.log2(row[mapping["S1"] + "_fc_Chpr"]), np.log2(row[mapping["S2"] + "_fc_Chpr"])):
                    if map_bias_check(row["Total_Counts_Humr"], row["Total_Counts_Chpr"]):
                        if row[0] != "chr20":
                            ch = row[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix] + row[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_humreffed" + suffix]
                            hh = row[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix] + row[mapping["S2"] + "_" + ct + "_Hyb2_Human_humreffed" + suffix]
                            cc = row[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix] + row[mapping["S2"] + "_" + ct + "_Hyb2_Chimp_chpreffed" + suffix]
                            hc = row[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix] + row[mapping["S2"] + "_" + ct + "_Hyb2_Human_chpreffed" + suffix]
                            chimp_pval = binom_test(hc, cc + hc)
                            human_pval = binom_test(hh, ch + hh)
                            out.append(list(row) + [chimp_pval, human_pval])
    df_filt_filt = pd.DataFrame(out)
    df_filt_filt.columns = list(df.columns) + ["Chimp binom_pval", "Human binom_pval"]
    if "nterpeak" in fold:
        df_filt_filt["Peak Humreffed"] = df_filt_filt["Peak"]
        df_filt_filt["Peak Chpreffed"] = df_filt_filt["Peak"]
    if fold == "All_Peaks":
        filee = "All_Peaks_Peaklist_Final_Anno_Humreffed_fixeded.bed"
    elif fold == "Down":
        filee = "All_Peaks_Peaklist_Down_Final_Anno_Humreffed_fixeded.bed"
    elif fold == "Interpeak":
        filee = "All_Peaks_Peaklist_Interpeak_Final_Anno_Humreffed_fixeded_del10.txt"
    elif fold == "Interpeak_Down":
        filee = "All_Peaks_Peaklist_Interpeak_Down_Final_Anno_Humreffed_fixeded_del10.txt"
    v = pd.read_csv(filee, sep = "\t", header = None).set_index(3)
    
    df_filt_filt_ind = df_filt_filt.set_index("Peak Humreffed")
    out = []
    for index, row in df_filt_filt_ind.iterrows():
        try:
            r = v.loc[index]
            if r[0] != "chr20":
                out.append([index] + list(row) + [r[0] + ":" + str(r[1]) + "-" + str(r[2])])
        except:
            pass
    dff = pd.DataFrame(out)
    dff.columns = ["Peak Humreffed"] + list(df_filt_filt_ind.columns) + ["Genomic location"]
    dff["Chimp FDR"] = fdrcorrection(dff["Chimp binom_pval"])[1]
    dff["Human FDR"] = fdrcorrection(dff["Human binom_pval"])[1]
    dff["Max FDR"] = np.max([dff["Chimp FDR"], dff["Human FDR"]], axis = 0)
    dff = dff.sort_values("Max FDR")
    dff = pd.DataFrame(dff[["Peak Humreffed", "Peak Chpreffed", "l2fc_Humr", "l2fc_Chpr", "mean_l2fc", "Chimp binom_pval", "Human binom_pval", "Genomic location", "Chimp FDR", "Human FDR", "Max FDR"]])
    #dff.to_csv("" + ct + "_ATAC_Filtered_all_peaks_new_new.txt", sep = "\t", index = False)
    #file = "" + ct + "_ATAC_Filtered_all_peaks_new_new.txt"
    if "Interpeak" not in fold:
        v = pd.read_csv("Correct_Gene_Names.txt", sep = "\t", header = None).set_index(0)
        lll = list(v.index)
        vv = dff
        out = []
        for index, row in vv.iterrows():
            new_row = list(row)
            #name = row["Peak Humreffed"].split("_")
            #name2 = row["Peak Chpreffed"].split("_")
            l = list(row)
            name = l[0]
            name2 = l[1]
            for key in fix_all_peaks.keys():
                if key in name:
                    name = name.replace(key, fix_all_peaks[key])
                if key in name2:
                    name2 = name2.replace(key, fix_all_peaks[key])
            if "enhancer" in name:
                name = name.split("_")
                gene1 = name[-4]
                gene2 = name[-2]
                if gene1 in list(v.index):
                    new_gene1 = v.loc[gene1][1]
                else:
                    new_gene1 = gene1
                if gene2 in list(v.index):
                    new_gene2 = v.loc[gene2][1]
                else:
                    new_gene2 = gene2
                name[-4] = new_gene1
                name[-2] = new_gene2
                l[0] = "_".join(name)
            elif "promoter" in name:
                ll = name.split("_")
                for i in ll:
                    if i in lll:
                        name = name.replace(i, v.loc[i][1])
                l[0] = name

            if "enhancer" in name2:
                name2 = name2.split("_")
                gene1 = name2[-4]
                gene2 = name2[-2]
                if gene1 in list(v.index):
                    new_gene1 = v.loc[gene1][1]
                else:
                    new_gene1 = gene1
                if gene2 in list(v.index):
                    new_gene2 = v.loc[gene2][1]
                else:
                    new_gene2 = gene2
                name2[-4] = new_gene1
                name2[-2] = new_gene2
                l[1] = "_".join(name2)
            elif "promoter" in name2:
                ll = name2.split("_")
                for i in ll:
                    if i in lll:
                        name2 = name2.replace(i, v.loc[i][1])
                l[1] = name2
            out.append(new_row)
        df = pd.DataFrame(out)
        df.columns = vv.columns
    if "nterpeak" in fold:
        df = dff
    file = ct + "_ATAC_Filtered_" + fold + "_fixed.txt"
    df.to_csv(file, sep = "\t", index = False)

    #Convert it to a bed file
    v = pd.read_csv(file, sep = "\t")
    out = []
    for index, row in v.iterrows():
        g = row["Genomic location"]
        l = [g.split(":")[0], g.split(":")[1].split("-")[0], g.split(":")[1].split("-")[1]] + list(row)
        out.append(l)
    df = pd.DataFrame(out)
    df.to_csv(file.replace("_TEST.txt", ".bed"), sep = "\t", header = False, index = False)

In [ ]:
#The analogous code when we have single replicate
def filter_sing(ct="HP", fold="All_Peaks", mapping={"S1":"ATAC16"}, suffix = "_all_peaks.txt"):
    df_hum = readin(ct, "umreffed", fold)
    df_chp = readin(ct, "hpreffed", fold)
    indices = []
    for index, row in df_hum.iterrows():
        if "promoter" in row["Peak"]:
            ad = row["Peak"].split("promoter")[0][:-1]
        else:
            ad = row["Peak"].split("enhancer")[0][:-1]
        indices.append(ad)
    df_hum.index = indices

    indices = []
    for index, row in df_chp.iterrows():
        if "promoter" in row["Peak"]:
            ad = row["Peak"].split("promoter")[0][:-1]
        else:
            ad = row["Peak"].split("enhancer")[0][:-1]
        indices.append(ad)
    df_chp.index = indices

    c = list(df_hum.columns)
    c[0] = "Peak Humreffed"
    df_hum.columns = c
    c = list(df_chp.columns)
    c[0] = "Peak Chpreffed"
    df_chp.columns = c
    df = df_hum.join(df_chp)
    print(df)
    df[mapping["S1"] + "_Human_Humr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix])
    df[mapping["S1"] + "_Chimp_Humr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix])
    df[mapping["S1"] + "_Human_Chpr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix])
    df[mapping["S1"] + "_Chimp_Chpr_CPM"] = 1000000*(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix]+1)/np.sum(df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix])

    df["Total_Counts_Humr"] = df[mapping["S1"] + "_Human_Humr_CPM"] + df[mapping["S1"] + "_Chimp_Humr_CPM"]
    df["Total_Counts_Chpr"] = df[mapping["S1"] + "_Human_Chpr_CPM"] + df[mapping["S1"] + "_Chimp_Chpr_CPM"]
    df[mapping["S1"] + "_fc_Humr"] = df[mapping["S1"] + "_Human_Humr_CPM"]/df[mapping["S1"] + "_Chimp_Humr_CPM"]
    df[mapping["S1"] + "_fc_Chpr"] = df[mapping["S1"] + "_Human_Chpr_CPM"]/df[mapping["S1"] + "_Chimp_Chpr_CPM"]
    df["l2fc_Humr"] = np.log2(df[mapping["S1"] + "_fc_Humr"])
    df["l2fc_Chpr"] = np.log2(df[mapping["S1"] + "_fc_Chpr"])
    df["mean_l2fc"] = (df["l2fc_Humr"] + df["l2fc_Chpr"])/2

    df["mean_counts_chimp_humr"] = df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix]
    df["mean_counts_human_humr"] = df[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix]
    df["mean_counts_chimp_chpr"] = df[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix]
    df["mean_counts_human_chpr"] = df[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix]

    out = []
    for index, row in df.iterrows():
        if (row["mean_counts_human_humr"] > 25 and row["mean_counts_human_chpr"] > 25) or (row["mean_counts_chimp_humr"] > 25 and row["mean_counts_chimp_chpr"] > 25):
            if not bias_check(np.log2(row[mapping["S1"] + "_fc_Humr"]), np.log2(row[mapping["S1"] + "_fc_Chpr"])):
                if map_bias_check(row["Total_Counts_Humr"], row["Total_Counts_Chpr"]):
                    if row[0] != "chr20":
                        ch = row[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_humreffed" + suffix]
                        hh = row[mapping["S1"] + "_" + ct + "_Hyb1_Human_humreffed" + suffix]
                        cc = row[mapping["S1"] + "_" + ct + "_Hyb1_Chimp_chpreffed" + suffix]
                        hc = row[mapping["S1"] + "_" + ct + "_Hyb1_Human_chpreffed" + suffix]
                        chimp_pval = binom_test(hc, cc + hc)
                        human_pval = binom_test(hh, ch + hh)
                        out.append(list(row) + [chimp_pval, human_pval])
    df_filt_filt = pd.DataFrame(out)
    df_filt_filt.columns = list(df.columns) + ["Chimp binom_pval", "Human binom_pval"]
    print(df_filt_filt)
    v = pd.read_csv("All_Peaks_Peaklist_Down_Final_Anno_Humreffed_fixeded.bed", sep = "\t", header = None).set_index(3)
    df_filt_filt_ind = df_filt_filt.set_index("Peak Humreffed")
    out = []
    for index, row in df_filt_filt_ind.iterrows():
        try:
            r = v.loc[index]
            if r[0] != "chr20":
                out.append([index] + list(row) + [r[0] + ":" + str(r[1]) + "-" + str(r[2])])
        except:
            pass
    dff = pd.DataFrame(out)
    dff.columns = list(df_filt_filt.columns) + ["Genomic location"]
    dff["Chimp FDR"] = fdrcorrection(dff["Chimp binom_pval"])[1]
    dff["Human FDR"] = fdrcorrection(dff["Human binom_pval"])[1]
    dff["Max FDR"] = np.max([dff["Chimp FDR"], dff["Human FDR"]], axis = 0)                                           
    dff = dff.sort_values("Max FDR")
    dff = pd.DataFrame(dff[["Peak Humreffed", "Peak Chpreffed", "l2fc_Humr", "l2fc_Chpr", "mean_l2fc", "Chimp binom_pval", "Human binom_pval", "Genomic location", "Chimp FDR", "Human FDR", "Max FDR"]])
    #dff.to_csv("" + ct + "_ATAC_Filtered_all_peaks_new_new.txt", sep = "\t", index = False)

    #file = "" + ct + "_ATAC_Filtered_all_peaks_new_new.txt"
    v = pd.read_csv("Correct_Gene_Names.txt", sep = "\t", header = None).set_index(0)
    lll = list(v.index)
    vv = dff
    out = []
    for index, row in vv.iterrows():
        new_row = list(row)
        #name = row["Peak Humreffed"].split("_")
        #name2 = row["Peak Chpreffed"].split("_")
        l = list(row)
        name = l[0]
        name2 = l[1]
        for key in fix_all_peaks.keys():
            if key in name:
                name = name.replace(key, fix_all_peaks[key])
            if key in name2:
                name2 = name2.replace(key, fix_all_peaks[key])
        if "enhancer" in name:
            name = name.split("_")
            gene1 = name[-4]
            gene2 = name[-2]
            if gene1 in list(v.index):
                new_gene1 = v.loc[gene1][1]
            else:
                new_gene1 = gene1
            if gene2 in list(v.index):
                new_gene2 = v.loc[gene2][1]
            else:
                new_gene2 = gene2
            name[-4] = new_gene1
            name[-2] = new_gene2
            l[0] = "_".join(name)
        elif "promoter" in name:
            ll = name.split("_")
            for i in ll:
                if i in lll:
                    name = name.replace(i, v.loc[i][1])
            l[0] = name

        if "enhancer" in name2:
            name2 = name2.split("_")
            gene1 = name2[-4]
            gene2 = name2[-2]
            if gene1 in list(v.index):
                new_gene1 = v.loc[gene1][1]
            else:
                new_gene1 = gene1
            if gene2 in list(v.index):
                new_gene2 = v.loc[gene2][1]
            else:
                new_gene2 = gene2
            name2[-4] = new_gene1
            name2[-2] = new_gene2
            l[1] = "_".join(name2)
        elif "promoter" in name2:
            ll = name2.split("_")
            for i in ll:
                if i in lll:
                    name2 = name2.replace(i, v.loc[i][1])
            l[1] = name2
        out.append(new_row)
    df = pd.DataFrame(out)
    df.columns = vv.columns
    file = ct + "_ATAC_Filtered_" + fold + "_fixed.txt"
    df.to_csv(file, sep = "\t", index = False)

    #Convert it to a bed file
    v = pd.read_csv(file, sep = "\t")
    out = []
    for index, row in v.iterrows():
        g = row["Genomic location"]
        l = [g.split(":")[0], g.split(":")[1].split("-")[0], g.split(":")[1].split("-")[1]] + list(row)
        out.append(l)
    df = pd.DataFrame(out)
    df.to_csv(file.replace(".txt", ".bed"), sep = "\t", header = False, index = False)

In [ ]:
filter_rep(ct="CM", fold="All_Peaks", mapping={"S1":"ATAC7", "S2":"ATAC8"}, suffix = "_all_peaks.txt")
filter_rep(ct="MN", fold="All_Peaks", mapping={"S1":"ATAC35", "S2":"ATAC36"}, suffix = "_all_peaks.txt")
filter_rep(ct="PP", fold="All_Peaks", mapping={"S1":"ATAC19", "S2":"ATAC20"}, suffix = "_all_peaks.txt")
filter_sing(ct="SKM", fold="All_Peaks", mapping={"S1":"ATAC11"}, suffix = "_all_peaks.txt")
filter_sing(ct="HP", fold="All_Peaks", mapping={"S1":"ATAC16"}, suffix = "_all_peaks.txt")